In [70]:
import glob, h5py, pickle, cv2, re, os
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import mat73
from IPython.display import clear_output
from scipy.io import loadmat


width = 224
height = 224
channels = 3

num_class = 100
sub = 'WLASL_' + str(num_class) + '_skel'
filename = '/mnt/HDD04/WASL Dataset/Scripts/datasets/'+sub+'.pkl'
print('File to be created: ' + filename)

File to be created: /mnt/HDD04/WASL Dataset/Scripts/datasets/WLASL_100_skel.pkl


In [71]:
env_path = '/mnt/HDD04/WASL Dataset/Scripts/envelopes_sim/'
env_threshold = 2e3
vid_path = '/mnt/HDD04/WASL Dataset/WLASL2000/'
skel_path = env_path.replace('envelopes_sim', 'skeletons_mat')
md_path = env_path.replace('envelopes_sim', 'microDoppler_sim')
json_path = env_path.replace('envelopes_sim', 'jsons')
proc_file_path = env_path.replace('envelopes_sim/', 'files' + str(num_class) + '.txt')
  
# removing the new line characters
with open(proc_file_path) as f:
    proc_files = [line.rstrip() for line in f]
print('Num file to process:', len(proc_files))

env_files = glob.glob(env_path + '*txt')
json_files = glob.glob(json_path + '*json')
print('Num. env files:', len(env_files))

Num file to process: 2038
Num. env files: 10461


In [72]:
import itertools, json
clas_dict = dict()
# for e in json_files:
#     #
#     fname = e.split('/')[-1]
#     underscore_idx = [m.start() for m in re.finditer('_', fname)]
#     word_id = fname.find('word')
#     clas = fname[word_id+4:underscore_idx[-1]]
#     if clas not in clas_dict:
#         clas_dict[clas] = 1
#     else:
#         clas_dict[clas] += 1
# sorted_clas_dict = {k: v for k, v in sorted(clas_dict.items(), key=lambda item: item[1])}
# reverse =dict(reversed(list(sorted_clas_dict.items())))
# with open(json_path) as ipf:
#     content = json.load(ipf)

for j in json_files:
#     gloss = c['gloss']
    with open(j) as ipf:
        content = json.load(ipf)
    if content['gloss'] in clas_dict:
        clas_dict[content['gloss']] += 1
    else:
        clas_dict[content['gloss']] = 1
#     for ins in c['instances']:
#         if not os.path.exists(env_path + ins['video_id'] + '.txt'):
#             continue
#         if gloss in clas_dict:
#             clas_dict[gloss] += 1
#         else:
#             clas_dict[gloss] = 1


# select the ones with most samples
# sorted_clas_dict = dict(itertools.islice(clas_dict.items(), num_class))
sorted_clas_dict = dict(sorted(clas_dict.items(), key=lambda item: item[1], reverse=True))
sorted_clas_dict = dict(itertools.islice(sorted_clas_dict.items(), num_class))
print(sorted_clas_dict)
select_classes = sorted_clas_dict.keys()
# print('select_classes:', select_classes)

{'book': 40, 'drink': 35, 'computer': 30, 'go': 26, 'before': 26, 'chair': 26, 'who': 25, 'clothes': 25, 'candy': 24, 'deaf': 23, 'cousin': 23, 'walk': 22, 'year': 22, 'help': 22, 'fine': 22, 'thin': 22, 'no': 22, 'yes': 22, 'all': 21, 'what': 21, 'thanksgiving': 21, 'mother': 21, 'orange': 21, 'finish': 21, 'cool': 21, 'many': 21, 'woman': 21, 'hot': 21, 'now': 21, 'table': 21, 'like': 21, 'black': 21, 'wrong': 20, 'bed': 20, 'blue': 20, 'fish': 20, 'shirt': 20, 'tall': 20, 'can': 20, 'study': 20, 'bowling': 20, 'man': 20, 'dog': 20, 'family': 20, 'hat': 20, 'hearing': 20, 'later': 20, 'white': 20, 'kiss': 20, 'graduate': 20, 'language': 20, 'pizza': 19, 'school': 19, 'eat': 19, 'secretary': 19, 'meet': 19, 'last': 19, 'want': 19, 'doctor': 19, 'dance': 19, 'play': 19, 'corn': 19, 'forget': 19, 'short': 19, 'accident': 19, 'give': 19, 'dark': 19, 'time': 19, 'bird': 19, 'pink': 19, 'color': 19, 'work': 19, 'cow': 19, 'change': 19, 'apple': 19, 'enjoy': 19, 'basketball': 18, 'full': 18

In [73]:
len(sorted_clas_dict)

100

In [74]:
j = json_files[0]
with open(json_path + '/12332.json') as ipf:
    content = json.load(ipf)
    print(content['gloss'])

computer


In [75]:
new_classes = dict()
for i in range(len(select_classes)):
    new_classes[list(select_classes)[i]] = i 
new_classes

{'book': 0,
 'drink': 1,
 'computer': 2,
 'go': 3,
 'before': 4,
 'chair': 5,
 'who': 6,
 'clothes': 7,
 'candy': 8,
 'deaf': 9,
 'cousin': 10,
 'walk': 11,
 'year': 12,
 'help': 13,
 'fine': 14,
 'thin': 15,
 'no': 16,
 'yes': 17,
 'all': 18,
 'what': 19,
 'thanksgiving': 20,
 'mother': 21,
 'orange': 22,
 'finish': 23,
 'cool': 24,
 'many': 25,
 'woman': 26,
 'hot': 27,
 'now': 28,
 'table': 29,
 'like': 30,
 'black': 31,
 'wrong': 32,
 'bed': 33,
 'blue': 34,
 'fish': 35,
 'shirt': 36,
 'tall': 37,
 'can': 38,
 'study': 39,
 'bowling': 40,
 'man': 41,
 'dog': 42,
 'family': 43,
 'hat': 44,
 'hearing': 45,
 'later': 46,
 'white': 47,
 'kiss': 48,
 'graduate': 49,
 'language': 50,
 'pizza': 51,
 'school': 52,
 'eat': 53,
 'secretary': 54,
 'meet': 55,
 'last': 56,
 'want': 57,
 'doctor': 58,
 'dance': 59,
 'play': 60,
 'corn': 61,
 'forget': 62,
 'short': 63,
 'accident': 64,
 'give': 65,
 'dark': 66,
 'time': 67,
 'bird': 68,
 'pink': 69,
 'color': 70,
 'work': 71,
 'cow': 72,
 'chan

In [76]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [77]:
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
    success, frame = vidObj.read() 
    # checks whether frames were extracted 
#     success = 1
    frames = []
    while success: 
  
        resized = cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR)
        success, frame = vidObj.read() 
#         print('Read a new frame: ', success)
        frames.append(resized) # .resized(width,height)
        count += 1
        if (count > (video_length-1)):
            break
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [78]:
train_files, test_files = train_test_split(proc_files, test_size=0.2, random_state=1)

In [79]:
train_images = []
train_vids = []
train_skels = []
train_labels = []
max_vid_len = 150

for i, e in enumerate(train_files):
    
    fname = e
    json_name = json_path + fname + '.json'
    with open(json_name) as ipf:
        content = json.load(ipf)
    
    if content['gloss'] not in new_classes:
        continue
    clas = new_classes[content['gloss']]
#     clas = clas_dict[content['gloss']]
    
#     if clas not in clas_dict:
#         print('skipping', clas, i)
#         continue
        
    # read envelope
    env_name = env_path + e + '.txt'
    if not os.path.exists(env_name):
        continue
    env = load_file(env_name)
    if sum(env[0]) < env_threshold:
        continue
        
    # read video
#     vid_name = vid_path + fname.replace('txt', 'avi')
#     vid = FrameCapture(vid_name)
#     num_frames = vid.shape[0]
#     train_vids.append(np.array(vid))
    
    # read skeleton
    skel_name = skel_path + fname + '.mat'
    skel = loadmat(skel_name)['hands']
    skel = np.transpose(skel, [2, 1, 0])
    skel = skel[3:]
    
    # pad missing frames
    if skel.shape[0] < max_vid_len:
        skel = np.concatenate([skel, np.tile(np.expand_dims(skel[0], 0), [max_vid_len - skel.shape[0], 1, 1])])
    elif skel.shape[0] > max_vid_len:
        skel = skel[:max_vid_len]
        
    train_skels.append(skel)
    
    # read microDoppler
#     md_name = md_path + fname.replace('txt', 'png')
#     img = cv2.imread(md_name)
#     img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC) # resize to (128,128)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Processing training file: '+ str(i+1) + '/' + str(len(train_files)))
#     train_images.append(img)
    train_labels.append(clas)

train_labels = to_categorical(np.array(train_labels), num_class)
# train_images = np.swapaxes(np.array(train_images), 1, 2).reshape(len(train_images), width, height, channels)/255.
# train_vids = np.array(train_vids, dtype=object)/255.
train_skels = np.array(train_skels)
# print(train_images.shape)
print(train_skels.shape)
print(train_labels.shape)
# print(train_vids.shape)

Processing training file: 1630/1630
(1567, 150, 3, 42)
(1567, 100)


In [80]:
maxlen = max([len(i) for i in train_skels])
maxlen

150

In [81]:
test_images = []
test_vids = []
test_skels = []
test_labels = []

for i, e in enumerate(test_files):
    
    fname = e
    json_name = json_path + fname + '.json'
    with open(json_name) as ipf:
        content = json.load(ipf)
    
    if content['gloss'] not in new_classes:
        continue
    clas = new_classes[content['gloss']]
#     clas = clas_dict[content['gloss']]
    
#     if clas not in clas_dict:
#         print('skipping', clas, i)
#         continue
        
    # read envelope
    env_name = env_path + e + '.txt'
    if not os.path.exists(env_name):
        continue
    env = load_file(env_name)
    if sum(env[0]) < env_threshold:
        continue
        
    # read video
#     vid_name = vid_path + fname.replace('txt', 'avi')
#     vid = FrameCapture(vid_name)
#     num_frames = vid.shape[0]
#     test_vids.append(np.array(vid))
    
    # read skeleton
    skel_name = skel_path + fname + '.mat'
    skel = loadmat(skel_name)['hands']
    skel = np.transpose(skel, [2, 1, 0])
    skel = skel[3:]
    
    # pad missing frames
    if skel.shape[0] < max_vid_len:
        skel = np.concatenate([skel, np.tile(np.expand_dims(skel[0], 0), [max_vid_len - skel.shape[0], 1, 1])])
    elif skel.shape[0] > max_vid_len:
        skel = skel[:max_vid_len]
    test_skels.append(skel)
    
    # read microDoppler
#     md_name = md_path + fname.replace('txt', 'png')
#     img = cv2.imread(md_name)
#     img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC) # resize to (128,128)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Processing training file: '+ str(i+1) + '/' + str(len(test_files)))
#     test_images.append(img)
    test_labels.append(clas)

test_labels = to_categorical(np.array(test_labels), num_class)
# test_images = np.swapaxes(np.array(test_images, dtype=object), 1, 2).reshape(len(test_images), width, height, channels)/255.
# test_vids = np.array(test_vids, dtype=object)/255.
test_skels = np.array(test_skels)
# print(test_images.shape)
print(test_skels.shape)
print(test_labels.shape)
# print(test_vids.shape)

Processing training file: 408/408
(393, 150, 3, 42)
(393, 100)


In [82]:
data = [train_skels, train_labels, 
        test_skels, test_labels]
with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

/mnt/HDD04/WASL Dataset/Scripts/datasets/WLASL_100_skel.pkl created.
